Import

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn import tree as _tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split #optional, depending on if you want to split your subgroups further.


Callable functions

In [1]:
def format_rct_dataset(dataset, outcome_field, treatment_field, interactions=True):
    """
    RCT format for Virtual Twins.

    Formats a dataset for Virtual Twins analysis.

    Checks and transforms the outcome, treatment, and predictor variables.

    Args:
        dataset (pd.DataFrame): DataFrame representing RCT's.
        outcome_field (str): Name of the outcome's field.
        treatment_field (str): Name of the treatment's field.
        interactions (bool): If running VirtualTwins with treatment's interactions, set to True (default).

    Returns:
        pd.DataFrame: Formatted DataFrame for Virtual Twins.
    """

    if not isinstance(dataset, pd.DataFrame):
        raise ValueError("Dataset parameter must be a pandas DataFrame")
    if not isinstance(outcome_field, str):
        raise ValueError(f"{outcome_field}, outcome_field parameter must be a string")
    if not isinstance(treatment_field, str):
        raise ValueError(f"{treatment_field}, treatment_field parameter must be a string")

    if outcome_field not in dataset.columns:
        raise ValueError(f"{outcome_field} must be in DataFrame columns")
    if treatment_field not in dataset.columns:
        raise ValueError(f"{treatment_field} must be in DataFrame columns")

    d = dataset.copy()  # Create a copy to avoid modifying the original

    outcome = d[outcome_field]
    if not isinstance(outcome, pd.Categorical):
        outcome = pd.Categorical(outcome)
    if len(outcome.categories) != 2:
        raise ValueError(f"outcome {outcome_field} must be binary")
    print(f'"{outcome.categories[1]}" will be the favorable outcome')
    d[outcome_field] = outcome

    treatment = d[treatment_field]
    if not pd.api.types.is_numeric_dtype(treatment):
        treatment = pd.to_numeric(treatment, errors='coerce') # convert to numeric, non numeric values become NaN
        d = d.dropna(subset=[treatment_field]) # drop rows where treatment field is NaN after conversion.
        treatment = d[treatment_field] # reassign treatment after dropping NA rows
    if not (len(treatment.unique()) == 2 and all(x in treatment.unique() for x in [0, 1])):
        raise ValueError(f"{treatment_field}, response must be numeric:\n 0 = no treatment \n 1 = treatment")
    d[treatment_field] = treatment

    predictors = [col for col in d.columns if col not in [outcome_field, treatment_field]]

    predictors_next = []
    for i in predictors:
        var = d[i]

        if pd.api.types.is_numeric_dtype(var) or pd.api.types.is_integer_dtype(var):
            predictors_next.append(i)

        if pd.api.types.is_string_dtype(var):
            var = pd.Categorical(var)
        if isinstance(var, pd.Categorical):
            if len(var.categories) > 2:
                if interactions:
                    print(f"Creation of dummy variables for {i}")
                    for l in var.categories:
                        n = f"{i}_{l}"
                        d[n] = (var == l).astype(int)
                        predictors_next.append(n)
                        print(f"Dummy variable {n} created")
                    # d = d.drop(columns=[i]) #not needed in python, as python doesnt need to delete the original categorical column.
                else:
                    if len(var.categories) > 32:
                        raise ValueError(f"{i} has too many levels (superior to 32)")
                    else:
                        print(f"Warning: {i} has more than 2 levels. Virtual Twins won't be able to run with interactions.")
                        predictors_next.append(i)
            elif len(var.categories) == 2:
                print(f"{i} is two-level factor. It has to be transformed into numeric value :")
                print(f"{var.categories[0]} becomes 0")
                print(f"{var.categories[1]} becomes 1")
                d[i] = (var == var.categories[1]).astype(int) # the second level is the positive level in python.
                predictors_next.append(i)
            else:
                print(f"{i} is deleted because only one level")

    colnames_order = [outcome_field, treatment_field] + predictors_next
    d = d[colnames_order]

    return d

Load Dataset

In [8]:
sepsis_data_python = pd.read_csv('dataset/sepsis_dataset.csv')
print(sepsis_data_python.head(10))

   survival  THERAPY  PRAPACHE     AGE  BLGCS  ORGANNUM    BLIL6     BLLPLAT  \
0         0        1        19  42.921     15         1   301.80  191.000000   
1         1        1        48  68.818     11         2   118.90  264.156481   
2         0        1        20  68.818     15         2    92.80  123.000000   
3         0        1        19  33.174     14         2  1232.00  244.000000   
4         0        1        48  46.532      3         4  2568.00   45.000000   
5         0        0        21  56.098     14         1   162.65  137.000000   
6         1        0        19  68.818     15         2  2568.00   45.000000   
7         0        1        19  46.532     15         3  4952.00   92.000000   
8         0        1        22  56.098     15         3   118.90  148.601978   
9         1        1        19  56.098     10         3  2568.00  109.000000   

    BLLBILI  BLLCREAT  TIMFIRST      BLADL  blSOFA  
0  2.913416  1.000000     17.17   0.000000    5.00  
1  0.400000  

Format to RCT

In [9]:
sepsisRCT = format_rct_dataset(sepsis_data_python, 'survival', 'THERAPY')

"1" will be the favorable outcome


Results

In [10]:
print(sepsisRCT.head(10))

  survival  THERAPY  PRAPACHE     AGE  BLGCS  ORGANNUM    BLIL6     BLLPLAT  \
0        0        1        19  42.921     15         1   301.80  191.000000   
1        1        1        48  68.818     11         2   118.90  264.156481   
2        0        1        20  68.818     15         2    92.80  123.000000   
3        0        1        19  33.174     14         2  1232.00  244.000000   
4        0        1        48  46.532      3         4  2568.00   45.000000   
5        0        0        21  56.098     14         1   162.65  137.000000   
6        1        0        19  68.818     15         2  2568.00   45.000000   
7        0        1        19  46.532     15         3  4952.00   92.000000   
8        0        1        22  56.098     15         3   118.90  148.601978   
9        1        1        19  56.098     10         3  2568.00  109.000000   

    BLLBILI  BLLCREAT  TIMFIRST      BLADL  blSOFA  
0  2.913416  1.000000     17.17   0.000000    5.00  
1  0.400000  1.100000   

Step 1:

Method: 
Simple Random Forest

In [57]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations

class VTObject:
    def __init__(self, data, treatment_col='THERAPY', outcome_col='survival'):
        self.data = data.copy()
        self.treatment_col = treatment_col
        self.outcome_col = outcome_col

    def getXwithInt(self):
        """Return predictors with interactions (T, X, X*T, X*(1-T))."""
        X = self.data.drop(self.outcome_col, axis=1)
        original_cols = X.columns.drop(self.treatment_col)  # Exclude treatment from original cols

        for col in original_cols:
            X[f"{col}*T"] = X[col] * X[self.treatment_col]
            X[f"{col}*(1-T)"] = X[col] * (1 - X[self.treatment_col])

        return X.drop(self.treatment_col, axis=1) # drop treatment column.

    def getX(self, interactions=True, trt=None):
        """Return predictors (T,X,X*T,X*(1-T)). Or (T,X) if interactions is FALSE.
            If trt is not None, return predictors for T = trt"""
        if trt is not None:
            return self.data[self.data[self.treatment_col] == trt].drop([self.outcome_col, self.treatment_col], axis=1)

        if interactions:
            return self.getXwithInt()

        return self.data.drop(self.outcome_col, axis=1).drop(self.treatment_col, axis=1)

    def getY(self, trt=None):
        """Return outcome. If trt is not None, return outcome for T = trt."""
        if trt is None:
            return self.data[self.outcome_col]

        return self.data[self.data[self.treatment_col] == trt][self.outcome_col]

    def switchTreatment(self):
        self.data[self.treatment_col] = 1 - self.data[self.treatment_col]

    def computeDelta(self):
        # Placeholder for delta computation. Implement as needed.
        pass

class VTDiffT:
    def __init__(self, vt_object, method="absolute"):
        self.vt_object = vt_object
        self.twin1 = np.full(len(vt_object.data), np.nan)  # Predictions if THERAPY = 1
        self.twin2 = np.full(len(vt_object.data), np.nan)  # Predictions if THERAPY = 0
        self.method = method
        self.difft = np.full(len(vt_object.data), np.nan)

        if method not in ["absolute", "relative", "logit"]:
            raise ValueError(f"Method {method} is not valid")

    def computeDifft(self):
        if np.isnan(self.twin1).any() or np.isnan(self.twin2).any():
            raise ValueError("Twins must be valid")

        treatment_col = self.vt_object.data[self.vt_object.treatment_col]

        if self.method == "absolute":
            self.difft = np.where(treatment_col == 1, self.twin1 - self.twin2, self.twin2 - self.twin1)
        elif self.method == "relative":
            self.difft = np.where(treatment_col == 1, self.twin1 / self.twin2, self.twin2 / self.twin1)
        elif self.method == "logit":
            def logit(x):
                return np.log(x / (1 - x))

            self.difft = np.where(treatment_col == 1, logit(self.twin1) - logit(self.twin2), logit(self.twin2) - logit(self.twin1))

        return self.difft

class VTForest(VTDiffT):
    def __init__(self, vt_object, model, interactions=False):
        super().__init__(vt_object)
        self.model = model
        self.interactions = interactions

    def checkModel(self, model):
        if not isinstance(model, RandomForestClassifier):
            raise ValueError("Model is not recognized. Must be RandomForestClassifier")

    def computeTwin1(self):
        X = self.vt_object.getX(interactions=self.interactions)
        self.twin1 = self.model.predict_proba(X)[:, 1]

    def computeTwin2(self):
        # print("Treatment column before switch inside computeTwin2:", self.vt_object.data['THERAPY'].values)
        self.vt_object.switchTreatment()
        # print("Treatment column after switch inside computeTwin2:", self.vt_object.data['THERAPY'].values)
        X_switched = self.vt_object.getX(interactions=self.interactions)
        # print("X after switch:", X_switched.head())  # Print the first few rows of X
        self.twin2 = self.model.predict_proba(X_switched)[:, 1]
        self.vt_object.switchTreatment() # Switch back.
        X_original = self.vt_object.getX(interactions=self.interactions)
        # print("X original:", X_original.head()) # print original X.


    def run(self):
        self.computeTwin1()
        self.computeTwin2()
        self.computeDifft()
        self.vt_object.computeDelta()  # Placeholder
        return self

    def getFullData(self):
        if len(self.twin1) != len(self.vt_object.data):
            raise ValueError("Twin1 must have same length as data")
        if len(self.twin2) != len(self.vt_object.data):
            raise ValueError("Twin2 must have same length as data")
        if len(self.difft) != len(self.vt_object.data):
            raise ValueError("Difft must have same length as data")

        tmp = self.vt_object.data.copy()
        tmp["twin1"] = self.twin1
        tmp["twin2"] = self.twin2
        tmp["difft"] = self.difft
        return tmp


def vt_forest(vt_object, model, interactions=True):
    vt_f_rf = VTForest(vt_object, model, interactions=interactions)
    return vt_f_rf.run()

# Example usage (assuming sepsisRCT is your DataFrame):
# np.random.seed(0)


vt_o = VTObject(sepsisRCT, treatment_col='THERAPY', outcome_col='survival')

model_rf = RandomForestClassifier(n_estimators=500, random_state=123)
model_rf.fit(vt_o.getX(interactions=True), vt_o.getY())

vt_f_rf = vt_forest(vt_o, model_rf, interactions=True)

full_data = vt_f_rf.getFullData()

# Print the predictions
print("Twin 1 Predictions (P1i):")
print(full_data["twin1"])

print("\nTwin 2 Predictions (P0i):")
print(full_data["twin2"])

print("\nDifft Predictions:")
print(full_data["difft"])

Twin 1 Predictions (P1i):
0     0.180
1     0.762
2     0.874
3     0.194
4     0.822
      ...  
95    0.186
96    0.210
97    0.862
98    0.710
99    0.170
Name: twin1, Length: 100, dtype: float64

Twin 2 Predictions (P0i):
0     0.734
1     0.460
2     0.436
3     0.564
4     0.594
      ...  
95    0.414
96    0.756
97    0.622
98    0.730
99    0.492
Name: twin2, Length: 100, dtype: float64

Difft Predictions:
0    -0.554
1    -0.302
2    -0.438
3    -0.370
4    -0.228
      ...  
95    0.228
96    0.546
97    0.240
98    0.020
99    0.322
Name: difft, Length: 100, dtype: float64


In [38]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from itertools import combinations

class VTObject:
    def __init__(self, data, treatment_col='THERAPY', outcome_col='survival'):
        self.data = data.copy()
        self.treatment_col = treatment_col
        self.outcome_col = outcome_col

    def getXwithInt(self):
        """Return predictors with interactions (T, X, X*T, X*(1-T))."""
        X = self.data.drop(self.outcome_col, axis=1)
        original_cols = X.columns.drop(self.treatment_col)  # Exclude treatment from original cols

        for col in original_cols:
            X[f"{col}*T"] = X[col] * X[self.treatment_col]
            X[f"{col}*(1-T)"] = X[col] * (1 - X[self.treatment_col])

        return X.drop(self.treatment_col, axis=1) # drop treatment column.

    def getX(self, interactions=True, trt=None):
        """Return predictors (T,X,X*T,X*(1-T)). Or (T,X) if interactions is FALSE.
            If trt is not None, return predictors for T = trt"""
        if trt is not None:
            return self.data[self.data[self.treatment_col] == trt].drop([self.outcome_col, self.treatment_col], axis=1)

        if interactions:
            return self.getXwithInt()

        return self.data.drop(self.outcome_col, axis=1).drop(self.treatment_col, axis=1)

    def getY(self, trt=None):
        """Return outcome. If trt is not None, return outcome for T = trt."""
        if trt is None:
            return self.data[self.outcome_col]

        return self.data[self.data[self.treatment_col] == trt][self.outcome_col]

    def switchTreatment(self):
        self.data[self.treatment_col] = 1 - self.data[self.treatment_col]

    def computeDelta(self):
        # Placeholder for delta computation. Implement as needed.
        pass

class VTDiffT:
    def __init__(self, vt_object, method="absolute"):
        self.vt_object = vt_object
        self.twin1 = np.full(len(vt_object.data), np.nan)
        self.twin2 = np.full(len(vt_object.data), np.nan)
        self.method = method
        self.difft = np.full(len(vt_object.data), np.nan)

        if method not in ["absolute", "relative", "logit"]:
            raise ValueError(f"Method {method} is not valid")

    def computeDifft(self):
        if np.isnan(self.twin1).any() or np.isnan(self.twin2).any():
            raise ValueError("Twins must be valid")

        treatment_col = self.vt_object.data[self.vt_object.treatment_col]

        if self.method == "absolute":
            self.difft = np.where(treatment_col == 1, self.twin1 - self.twin2, self.twin2 - self.twin1)
        elif self.method == "relative":
            self.difft = np.where(treatment_col == 1, self.twin1 / self.twin2, self.twin2 / self.twin1)
        elif self.method == "logit":
            def logit(x):
                return np.log(x / (1 - x))

            self.difft = np.where(treatment_col == 1, logit(self.twin1) - logit(self.twin2), logit(self.twin2) - logit(self.twin1))

        return self.difft
    
class SimpleRandomForest:
    def __init__(self, n_trees=10, max_depth=3):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            indices = np.random.choice(len(X), len(X), replace=True)
            tree = SimpleDecisionTree(max_depth=self.max_depth)
            tree.tree = tree.fit(X[indices], y[indices])
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for x in X:
            tree_predictions = [tree.predict(x) for tree in self.trees]
            predictions.append(np.mean(tree_predictions))
        return np.array(predictions)


class VTForest(VTDiffT):
    def __init__(self, vt_object, model, interactions=True):
        super().__init__(vt_object)
        self.model = model
        self.interactions = interactions

    def checkModel(self, model):
        if not isinstance(model, RandomForestRegressor):
            raise ValueError("Model is not recognized. Must be RandomForestRegressor")

    def computeTwin1(self):
        X = self.vt_object.getX(interactions=self.interactions)
        self.twin1 = self.model.predict(X) # Change to predict

    def computeTwin2(self):
        self.vt_object.switchTreatment()
        X = self.vt_object.getX(interactions=self.interactions)
        self.twin2 = self.model.predict(X) # Change to predict
        self.vt_object.switchTreatment() # Switch back

    def run(self):
        self.computeTwin1()
        self.computeTwin2()
        self.computeDifft()
        self.vt_object.computeDelta()  # Placeholder
        return self

    def getFullData(self):
        if len(self.twin1) != len(self.vt_object.data):
            raise ValueError("Twin1 must have same length as data")
        if len(self.twin2) != len(self.vt_object.data):
            raise ValueError("Twin2 must have same length as data")
        if len(self.difft) != len(self.vt_object.data):
            raise ValueError("Difft must have same length as data")

        tmp = self.vt_object.data.copy()
        tmp["twin1"] = self.twin1
        tmp["twin2"] = self.twin2
        tmp["difft"] = self.difft
        return tmp

def vt_forest(vt_object, model, interactions=True):
    vt_f_rf = VTForest(vt_object, model, interactions=interactions)
    return vt_f_rf.run()


vt_o = VTObject(sepsisRCT, treatment_col='THERAPY', outcome_col='survival')

model_rf = RandomForestRegressor(n_estimators=500, random_state=123)
model_rf.fit(vt_o.getX(interactions=True), vt_o.getY())

vt_f_rf = vt_forest(vt_o, model_rf, interactions=True)

full_data = vt_f_rf.getFullData()

# Print the predictions
print("Twin 1 Predictions (P1i):")
print(full_data["twin1"])

print("\nTwin 2 Predictions (P0i):")
print(full_data["twin2"])

print("\nDifft Predictions:")
print(full_data["difft"])

Twin 1 Predictions (P1i):
0     53.235387
1     62.696990
2     60.377392
3     49.287797
4     41.936139
        ...    
95    31.127134
96    49.609537
97    14.718920
98    70.267194
99    26.110533
Name: twin1, Length: 100, dtype: float64

Twin 2 Predictions (P0i):
0     45.309247
1     56.046100
2     59.614659
3     63.863781
4     65.170122
        ...    
95    61.924583
96    67.352592
97    58.339080
98    57.139189
99    33.498386
Name: twin2, Length: 100, dtype: float64

Difft Predictions:
0      7.926140
1     -6.650890
2      0.762733
3     14.575984
4     23.233983
        ...    
95    30.797449
96    17.743055
97    43.620160
98    13.128005
99    -7.387853
Name: difft, Length: 100, dtype: float64


In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def vt_predict(model: RandomForestRegressor, newdata: pd.DataFrame, treatment_field: str = None, counterfactual: bool = False):
    """
    Predicts outcomes using the provided model.
    
    Parameters:
      model: A trained RandomForestRegressor.
      newdata: A DataFrame of features for which to make predictions.
      treatment_field: The name of the treatment column in newdata.
      counterfactual: If True, set the treatment_field to 0 for a counterfactual prediction.
    
    Returns:
      Predictions (numpy array) from the model.
    """
    data = newdata.copy()
    if counterfactual and treatment_field is not None:
        data[treatment_field] = 0  # Set treatment to 0 for counterfactual prediction
    return model.predict(data)

# Define outcome and treatment fields
outcome_field = "survival"   # Change as needed
treatment_field = "THERAPY"  # Change as needed

# Assume sepsisRCT is your DataFrame containing the data
# Prepare the feature matrix (X) and target variable (y)
X = sepsisRCT.drop(columns=[outcome_field])  # Keep treatment in X
y = sepsisRCT[outcome_field]                  # Target variable (e.g., survival outcome)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Initialize and train the Random Forest model
model = RandomForestRegressor(n_estimators=500, random_state=123)
model.fit(X_train, y_train)

# Make predictions using vt_predict
P1 = vt_predict(model, X_test, treatment_field=treatment_field, counterfactual=False)
P0 = vt_predict(model, X_test, treatment_field=treatment_field, counterfactual=True)

# Evaluate the model
mse = mean_squared_error(y_test, P1)
print(f"Model Mean Squared Error: {mse:.4f}")

# Print P1 and P0
print("P1 (Predicted with treatment):", P1)
print("P0 (Predicted without treatment):", P0)


Model Mean Squared Error: 838.2919
P1 (Predicted with treatment): [41.8143453  44.99009273 47.84048398 49.31955403 28.7526053  46.65189319
 41.44399994 44.59344994 45.59229559 57.75639252 52.72564858 51.01804228
 55.34715392 31.47883471 48.34156011 36.28400419 48.17222085 50.486273
 57.83187388 26.2265777 ]
P0 (Predicted without treatment): [41.8143453  42.06912802 47.84048398 48.0665411  28.7526053  45.41760561
 38.76044241 43.73846882 45.0528544  57.75639252 51.60644279 49.70503991
 53.9586119  29.68493072 46.63228638 36.28400419 48.17222085 50.85002149
 57.22233083 26.2265777 ]


Check results

In [50]:
# Create a Pandas DataFrame for display
probabilities_df = pd.DataFrame({
    'P0': P0,
    'P1': P1
})

# Add an Observation column
probabilities_df['Observation'] = range(1, len(probabilities_df) + 1)

# Reorder columns
probabilities_df = probabilities_df[['Observation', 'P0', 'P1']]

# Display the DataFrame
print(probabilities_df)


     Observation     P0     P1
0              1  0.480  0.144
1              2  0.476  0.942
2              3  0.406  0.092
3              4  0.338  0.050
4              5  0.428  0.124
..           ...    ...    ...
465          466  0.136  0.332
466          467  0.692  0.198
467          468  0.156  0.364
468          469  0.164  0.324
469          470  0.112  0.792

[470 rows x 3 columns]


Step 2

using SRF and only Classification Tree because tree.class used in subgroups in cran 